In [ ]:
import numpy as np
import cv2
import pywt
from skimage.metrics import structural_similarity as ssim

def extract_patches(image, patch_size=(32, 32), stride=16):
    """
    Extract patches from the input image.
    """
    patches = []
    height, width = image.shape
    for y in range(0, height - patch_size[0] + 1, stride):
        for x in range(0, width - patch_size[1] + 1, stride):
            patch = image[y:y+patch_size[0], x:x+patch_size[1]]
            patches.append(patch)
    return patches

def fft_magnitude(patch):
    """
    Compute the magnitude of the 2D Fourier transform and take its logarithm.
    """
    f_transform = np.fft.fft2(patch)
    f_transform = np.fft.fftshift(f_transform)
    magnitude = np.abs(f_transform)
    return np.log(1 + magnitude)

def wavelet_transform(patch):
    """
    Compute the approximation coefficients of the 2D Wavelet transform.
    """
    coeffs = pywt.wavedec2(patch, 'haar')
    return coeffs[0]

def compare_noise(patch1, patch2):
    """
    Compare noise patterns between two patches using SSIM.
    """
    return ssim(patch1, patch2)

def detect_change(patch1, patch2, threshold=0.5):
    """
    Detect changes in noise distribution by comparing patches.
    """
    difference = compare_noise(patch1, patch2)
    return difference > threshold

def detect_modifications(image, patch_size=(32, 32), stride=16, threshold=0.5):
    """
    Detect potential modifications in the scanned invoice image.
    """
    grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    patches = extract_patches(grayscale_image, patch_size, stride)

    modifications = []

    for i, patch1 in enumerate(patches):
        for j, patch2 in enumerate(patches):
            if i != j:
                if detect_change(patch1, patch2, threshold):
                    modifications.append((i, j))

    return modifications

image_path = '/home/acer/Downloads/Ratnadeep-Bill-Modified.jpg'
image = cv2.imread(image_path)

# Set parameters and detect modifications
patch_size = (32, 32)
stride = 16
threshold = 0.5
modifications = detect_modifications(image, patch_size, stride, threshold)

# Print the indices of potentially modified patches
for mod in modifications:
    print(f"Patches {mod[0]} and {mod[1]} may contain modifications.")
